[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
# cобираем айди (циклом по 100)
unik=[]
limit_ = 100
for i in tqdm(range (1,10700,100)):
    offset_ = i
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    unik.extend(objids)
    #print(i)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/107 [00:00<?, ?it/s]

In [4]:
#нормализуем и записывем инфо
dfs = pd.DataFrame()
for ids in tqdm(unik):
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' + str(ids)
    res = requests.get(url)
    dfs=dfs.append(pd.json_normalize(res.json().get('data')))
    #print(unik.index(ids))
dfs.to_excel('stop_loc.xlsx')#записываем в иксель
dfs.to_pickle("./dummy.pkl")#записываем в пикл

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/10582 [00:00<?, ?it/s]

In [5]:
#записываем в БД
import sqlite3 as sl
con = sl.connect('my-test.db')
dfs1 = dfs.applymap(str)
dfs1.to_sql('dfs1', con)

Задание*

In [6]:
import os

In [38]:
#делаем табличку с ссылками на картинки
dfs1 = pd.DataFrame()
for ids in tqdm(unik):
    url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' + str(ids)
    res = requests.get(url)
    dfs1=dfs1.append(pd.json_normalize(res.json().get('data').get('photoRenderDTO')))

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/10582 [00:00<?, ?it/s]

In [52]:
#цикл для сохранения картинок в папке с айди стройки
for i, row in tqdm(dfs1[['objId','objRenderPhotoUrl']].iterrows()):
    if not os.path.exists(str(row['objId'])):
        os.makedirs(str(row['objId']))
        urlimg=row['objRenderPhotoUrl']
        res2=requests.get(urlimg)
        imageFile=open(os.path.join(str(row['objId']),str(i)),'wb')
        for chunk in res2.iter_content(100000):
            imageFile.write(chunk)
        imageFile.close()
        #imageFile.write(row['objRenderPhotoUrl'].content)
        #imageFile.close()
    else:
        urlimg=row['objRenderPhotoUrl']
        res2=requests.get(urlimg)
        imageFile=open(os.path.join(str(row['objId']),str(i)),'wb')
        for chunk in res2.iter_content(100000):
            imageFile.write(chunk)
        imageFile.close()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


0it [00:00, ?it/s]

KeyboardInterrupt: 